In [2]:
! pip install folium


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import folium
import csv
import pandas as pd

In [4]:
df = pd.read_csv('tripdata.csv')
start_stations = df.groupby(['start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude']).size().reset_index(name='num_rides')
df.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
0,52185,32:10.1,01:56.0,21.0,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,13.0,Commercial St at Montgomery St,37.794231,-122.402923,4902,Customer,1984.0,Male,No
1,42521,53:21.8,42:03.1,23.0,The Embarcadero at Steuart St,37.791464,-122.391034,81.0,Berry St at 4th St,37.775880,-122.393170,2535,Customer,NaN,NaN,No
2,61854,13:13.2,24:08.1,86.0,Market St at Dolores St,37.769305,-122.426826,3.0,Powell St BART Station (Market St at 4th St),37.786375,-122.404904,5905,Customer,1972.0,Male,No
3,36490,54:26.0,02:36.8,375.0,Grove St at Masonic Ave,37.774836,-122.446546,70.0,Central Ave at Fell St,37.773311,-122.444293,6638,Subscriber,1989.0,Other,No
4,1585,54:18.5,20:44.1,7.0,Frank H Ogawa Plaza,37.804562,-122.271738,222.0,10th Ave at E 15th St,37.792714,-122.248780,4898,Subscriber,1974.0,Male,Yes


In [5]:
#Rename columns for consistency
start_stations.columns = ['station_id', 'station_name', 'latitude', 'longitude', 'num_rides']
df.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
0,52185,32:10.1,01:56.0,21.0,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,13.0,Commercial St at Montgomery St,37.794231,-122.402923,4902,Customer,1984.0,Male,No
1,42521,53:21.8,42:03.1,23.0,The Embarcadero at Steuart St,37.791464,-122.391034,81.0,Berry St at 4th St,37.775880,-122.393170,2535,Customer,NaN,NaN,No
2,61854,13:13.2,24:08.1,86.0,Market St at Dolores St,37.769305,-122.426826,3.0,Powell St BART Station (Market St at 4th St),37.786375,-122.404904,5905,Customer,1972.0,Male,No
3,36490,54:26.0,02:36.8,375.0,Grove St at Masonic Ave,37.774836,-122.446546,70.0,Central Ave at Fell St,37.773311,-122.444293,6638,Subscriber,1989.0,Other,No
4,1585,54:18.5,20:44.1,7.0,Frank H Ogawa Plaza,37.804562,-122.271738,222.0,10th Ave at E 15th St,37.792714,-122.248780,4898,Subscriber,1974.0,Male,Yes


In [8]:
# Analyze the bike share data:
most_popular_station = start_stations.loc[start_stations['num_rides'].idxmax()]
least_popular_station = start_stations.loc[start_stations['num_rides'].idxmin()]
average_rides_per_station = start_stations['num_rides'].mean()

print("Most popular station:", most_popular_station['station_name'], "with", most_popular_station['num_rides'], "rides")
print("Least popular station:", least_popular_station['station_name'], "with", least_popular_station['num_rides'], "rides")
print("Average number of rides per station:", average_rides_per_station)

Most popular station: Market St at 10th St with 3904 rides
Least popular station: 16th St Depot with 2 rides
Average number of rides per station: 556.8844984802431


In [9]:
import folium

map_center = [37.77, -122.42]  # San Francisco, CA
bike_map = folium.Map(location=map_center, zoom_start=12)

# Plot the bike share station data on the map:

def get_marker_color(num_rides):
    if num_rides < 1000:
        return 'green'
    elif num_rides < 5000:
        return 'orange'
    else:
        return 'red'


for _, row in start_stations.iterrows():
    
    # Get the appropriate marker color based on the number of rides for the current station
    marker_color = get_marker_color(row['num_rides'])
    
    # Create a folium Marker object with the station's latitude and longitude coordinates
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],  # Marker location
        icon=folium.Icon(color=marker_color),  # Custom marker color based on the number of rides
        # Set the popup content to display the station name and number of rides
        popup=f"Station: {row['station_name']}<br>Rides: {row['num_rides']}"
    )
    
    # Add the marker to the bike_map
    marker.add_to(bike_map)


# Save the folium map as an HTML file
bike_map.save('bike_map.html')


In [10]:
bike_map